# AAG - Titanic
## Mahouin Julien

In [8]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
import pandas as pd
import numpy as np
import math
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
# Méthode de création des intervalles pour la discrétisation du jeu de données
def discretisation_dataset(dataset, j):
    # Calcul de la moyenne de la colonne
    moy = 0
    for i in range(dataset.shape[0]):
        moy += dataset[i][j]
    moy /= dataset.shape[0]
    
    # Discrétisation des données par rapport à cette valeur
    for i in range(dataset.shape[0]):
        if(dataset[i][j] >= moy):
            dataset[i][j] = 1
        else:
            dataset[i][j] = 0
            
def discretiser_sexe(dataset):
    for i in range(dataset.shape[0]):
        if(dataset[i][2] == 'male'):
            dataset[i][2] = 1
        else:
            dataset[i][2] = 0

In [3]:
# Méthode de création de la table des fréquences des features par classe
def creerTableFrequences(dataset):
    # Calcul des fréquences de chaque classe pour chaque feature
    freq_classes_features = [] # La liste des matrices de fréquences
    
    # Calcul des fréquences pour la première feature
    freq_classes = np.zeros((3, 2))
    for i in range(dataset.shape[0]):
        freq_classes[dataset[i][1]-1][dataset[i][0]] += 1
    freq_classes_features.append(freq_classes)
        
    # Calcul des fréquences pour les features suivantes
    for j in range(2, dataset.shape[1]):
        freq_classes = np.zeros((2, 2))
        for i in range(dataset.shape[0]):
            freq_classes[dataset[i][j]][dataset[i][0]] += 1
        freq_classes_features.append(freq_classes)
    
    # Retour de l'ensemble des tables de fréquences des features par classe
    return freq_classes_features

## k-ppv

In [13]:
# Méthode K-ppv
def kppv_oneData(data, dataset, k):
    # Calcul des distances entre la donnée de test et l'ensemble des données d'apprentissage selon la formule de la
    # distance de Manhattan
    distances = np.zeros(dataset.shape[0])
    for i in range(dataset.shape[0]):
        for j in range(1, dataset.shape[1]):
            #if not(math.isnan(data[j])) and not(math.isnan(dataset[i, j])):
            distances[i] += np.absolute(data[j] - dataset[i, j])
            # Calcul de la distance Euclidienne
            #distances[i] += np.square(np.absolute(data[j] - dataset[i, j]))
        #distances[i] = np.sqrt(distances[i])
        
    # Conservation des rangs des k plus proches voisins
    rangs = distances.argsort()
    k_plus_proches_voisins = rangs[:k]
    
    # Calcul de la fréquences des classes parmis les k plus proches voisins
    freq_classes = np.zeros((2,))
    for i in range(k):
        freq_classes[dataset[k_plus_proches_voisins[i]][0]] += 1
    
    # Retour de la classe la plus fréquente de ses voisins
    return int(np.argmax(freq_classes))

def kppv(datas, dataset, k):
    datasClassees = np.zeros(datas.shape[0], dtype=int)
    
    for i in range(datas.shape[0]):
        datasClassees[i] = kppv_oneData(datas[i], dataset, k)
    
    return datasClassees

# Méthode Naïve Bayes avec retour des mesures d'évaluation
def kppvV2(dataset_test, dataset_apprentissage, k):
    # Table comprenant les résultats de prédictions Références/Hypothèses, initialisée avec des 1 pour éviter les 0 lors des
    # calculs
    table_result = np.ones([2, 2])
    sauv_result = [] # Sauvegarde des résultats brutes pour chaque données
    # Prédiction de chacune des données et affectation dans la table
    resultat = kppv(dataset_test, dataset_apprentissage, k)
    sauv_result.append(resultat)
    for i in range(dataset_test.shape[0]):
        table_result[resultat[i], dataset_apprentissage[i][0]] += 1
        
    
    # Calcul des prédictions et des mesures d'évaluation
    return [sauv_result, accuracy(table_result), precisions(table_result), rappels(table_result)]

## Main

In [17]:
# Préparation du jeu d'apprentissage
spreadsheet = pd.read_csv('train.csv')
dataTrain = spreadsheet.to_numpy()
dataTrain = np.delete(dataTrain, [0, 3, 5, 8, 9, 10, 11], 1)

discretiser_sexe(dataTrain)

# Préparation du jeu de test
spreadsheet = pd.read_csv('test.csv')
dataTests = spreadsheet.to_numpy()
dataTest = np.delete(dataTests, [0, 2, 4, 7, 8, 9, 10], 1)
# Ajout de la colonne des classes
col = np.zeros((dataTest.shape[0],1), dtype=int)
dataTest = np.hstack((col, dataTest))

discretiser_sexe(dataTest)

#k = int(dataTrain.shape[0] / 3)
k = int(np.sqrt(dataTrain.shape[0]))

predictions = kppv(dataTest, dataTrain, k)

# Concaténation des ID et des prédictions au format DataFrame
df = pd.DataFrame({'PassengerId': dataTests[:, 0],
                   'Survived': predictions})
#df.drop([0], axis=1)
#df.drop(columns = df.columns[0], axis = 1, inplace= True)
# Exportation des résultats au format csv
from pathlib import Path  
#filepath = Path('Resultats/out.csv')  
#filepath.parent.mkdir(parents=True, exist_ok=True)  
#df.to_csv(filepath) 
df.to_csv(r'Resultats/out.csv', index=False, header=True)

res, acc, prec, rapp = kppvV2(dataTrain, dataTrain, k)
#print('--->Prédictions :\n', res)
print('--->Accuracy :\n', acc)
print('--->Precisions :\n', prec)
print('--->Rappels :\n', rapp)

#--->Accuracy :
# 0.8055865921787709
#--->Precisions :
# [0.91651543 0.62790698]
#--->Rappels :
# [0.79778831 0.82442748]
# 0.77033

[[0 3 'male' 1 0]
 [1 1 'female' 1 0]
 [1 3 'female' 0 0]
 ...
 [0 3 'female' 1 2]
 [1 1 'male' 0 0]
 [0 3 'male' 0 0]]
--->Accuracy :
 0.8055865921787709
--->Precisions :
 [0.91651543 0.62790698]
--->Rappels :
 [0.79778831 0.82442748]


## Mesures d'évaluation

In [14]:
# Méthode de calcul de l'accuracy
def accuracy(resultats):
    nb_bien_classees = 0
    for i in range(resultats.shape[0]):
        nb_bien_classees += resultats[i,i]
    return nb_bien_classees / resultats.sum()

# Méthode de calcul de la précision
def precisions(resultats):
    precisions = np.zeros(resultats.shape[0])
    for i in range(resultats.shape[0]):
        precisions[i] = resultats[i, i] / resultats.sum(axis=0)[i]
    return precisions

# Méthode de calcul du rappel
def rappels(resultats):
    rappels = np.zeros(resultats.shape[0])
    for i in range(resultats.shape[0]):
        rappels[i] = resultats[i, i] / resultats.sum(axis=1)[i]
    return rappels